In [23]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [3]:
#Load CSV file into dfs
orders = pd.read_csv('data/orders.csv', sep=';')
items = pd.read_csv('data/items.csv', sep=';')

#### 1. Data Exploration

##### 1.1. Orders Table

In [15]:
orders.head()

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,department,channel,owner,site,CreatedAt
0,2000093387,2020-04-24 00:00:00,GOPAY_CARD,S101,1,NaN,3.506048,0.000000,3.496395,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2020-04-24 21:50:00
1,2000093387,2020-04-24 00:00:00,GOPAY_CARD,S113,1,NaN,-0.705913,0.000000,-0.717209,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2020-04-24 21:50:00
2,2000093388,2020-04-24 00:00:00,COD,ZB00089178,1,NaN,5.829845,2.209302,6.472868,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00
3,2000093388,2020-04-24 00:00:00,COD,ZB00138060,1,NaN,5.829845,2.209302,6.472868,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00
4,2000093388,2020-04-24 00:00:00,COD,ZB00015664,1,NaN,19.379845,18.731008,32.015504,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00


In [5]:
#Number of rows and columns
orders.shape

(2128524, 14)

In [6]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2128524 entries, 0 to 2128523
Data columns (total 14 columns):
 #   Column               Dtype  
---  ------               -----  
 0   order_id             int64  
 1   date                 object 
 2   payment              object 
 3   item_code            object 
 4   quantity             int64  
 5   gift_quantity        float64
 6   unit_price_vat_excl  float64
 7   unit_cogs            float64
 8   unit_rrp_vat_excl    float64
 9   department           object 
 10  channel              object 
 11  owner                object 
 12  site                 object 
 13  CreatedAt            object 
dtypes: float64(4), int64(2), object(8)
memory usage: 227.4+ MB


In [7]:
orders.isnull().sum()

order_id                     0
date                         0
payment                    567
item_code                    0
quantity                     0
gift_quantity          2118134
unit_price_vat_excl          0
unit_cogs                    0
unit_rrp_vat_excl           98
department                   0
channel                      0
owner                        0
site                         0
CreatedAt                    0
dtype: int64

* Relevant columns: order_id, date, item_code, and site (to identify countries)

* gift_quantity column shows that there were only 10,390 items given as gifts.

##### Dataset date range:

In [8]:
print( 'Dataset orders starts on', orders['date'].min(), 'and ends on', orders['date'].max())


Dataset orders starts on 2019-06-01 00:00:00 and ends on 2022-01-14 00:00:00


In [9]:
#Number of items ordered by item_code
orders['item_code'].value_counts().head(8)

S101          406708
S113          347381
S114           15649
ZB00077277      2398
ZB00053597      2070
ZB00025998      1941
ZB00025834      1773
ZB00026748      1595
Name: item_code, dtype: int64

##### The top 5 most sold items are S101, S113, S114, ZB00077277, and ZB00053597.

In [10]:
Top5 = items[items['item_code']=='S101'].append(items[items['item_code']=='S113']).append(items[items['item_code']=='S114']).append(items[items['item_code']=='ZB00077277']).append(items[items['item_code']=='ZB00053597'])
Top5

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
306868,S101,Dopravné,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307919,S113,Platba,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308992,S114,Sleva,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285821,ZB00077277,Nike Everyday 3 pack,sx7676-100,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,football,men,adults,White,M
269154,ZB00053597,Nike Everyday 3 pack,sx7676-100,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,football,men,adults,White,L


#### Translations: 
- item_name: Dopravné = *Shipping*
- item_name: Platba = *Payment*
- item_name: Sleva = *Discount*
- bran_name: žádný výrobce = *No Manufacturer*

After discarding shipping, payment and discount as part of the top 5 products, the new top 5 is the following:

In [88]:
Top5=items[items['item_code']=='ZB00077277'].append(items[items['item_code']=='ZB00053597']).append(items[items['item_code']=='ZB00025998']).append(items[items['item_code']=='ZB00025834']).append(items[items['item_code']=='ZB00026748'])
Top5

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
285821,ZB00077277,Nike Everyday 3 pack,sx7676-100,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,football,men,adults,White,M
269154,ZB00053597,Nike Everyday 3 pack,sx7676-100,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,football,men,adults,White,L
178949,ZB00025998,3PPK VALUE COTTON CREW-SMLX,sx4508-101,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,fitness,unisex,adults,White,M
181425,ZB00025834,3PPK VALUE COTTON CREW-SMLX,sx4508-965,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,fitness,unisex,adults,White,M
190826,ZB00026748,3PPK VALUE COTTON CREW-SMLX,sx4508-965,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,fitness,unisex,adults,White,L


As can be seen, the top 5 sold product is compused by socks of the brand Nike.

Next 5 products (After the top 5):

In [102]:
orders['item_code'].value_counts().head(13)

S101          406708
S113          347381
S114           15649
ZB00077277      2398
ZB00053597      2070
ZB00025998      1941
ZB00025834      1773
ZB00026748      1595
ZB00051626      1526
ZB00088644      1419
ZB00040759      1306
ZB00013122      1169
ZB00030275       993
Name: item_code, dtype: int64

In [103]:
Next5 = items[items['item_code']=='ZB00051626'].append(items[items['item_code']=='ZB00088644']).append(items[items['item_code']=='ZB00040759']).append(items[items['item_code']=='ZB00013122']).append(items[items['item_code']=='ZB00030275'])
Next5

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
266160,ZB00051626,3PPK VALUE COTTON CREW-SMLX,sx4508-101,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,fitness,unisex,adults,White,L
79331,ZB00088644,M NK DRY PARK20 PANT KP,bv6877-010,84,Nike,200.0,Apparel,240.0,Pants,0.0,NaN,football,men,adults,Black,M
260049,ZB00040759,M NP SHORT,bv5635-010,84,Nike,200.0,Apparel,240.0,Pants,242.0,Shorts,fitness,men,adults,Black,M
289407,ZB00013122,M NK DRY PARK 1STLYR JSY LS,av2609-010,84,Nike,200.0,Apparel,230.0,T-Shirts,233.0,Longsleeved (LS) shirts,football,men,adults,Black,M
199913,ZB00030275,NK STRKE SNOOD,bq5832-013,84,Nike,300.0,Equipment,299.0,Other Apparel,0.0,NaN,football,unisex,adults,Black,S/M


In [51]:
#Check duplicates
orders[orders.duplicated()]

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,department,channel,owner,site,CreatedAt
164348,1900092622,2019-07-01 00:00:00,COD,ZB00018246,1,NaN,57.627132,44.215891,96.066279,E-COMMER,RUN.czC,VO_B_RUO,top4running.cz,2019-07-01 21:46:00
164349,1900092622,2019-07-01 00:00:00,COD,S101,1,NaN,2.850775,0.000000,2.850775,E-COMMER,RUN.czC,VO_B_RUO,top4running.cz,2019-07-01 21:46:00
164350,1900092622,2019-07-01 00:00:00,COD,S113,1,NaN,0.929070,0.000000,0.929070,E-COMMER,RUN.czC,VO_B_RUO,top4running.cz,2019-07-01 21:46:00
165359,1900093069,2019-07-02 00:00:00,COD,ZB00013773,1,NaN,133.570880,111.286822,190.808641,E-COMMER,FTB.huC,FTB.huO,11teamsports.hu,2019-07-02 13:02:00
165381,1900093077,2019-07-02 00:00:00,COD,ZB00004598,1,NaN,80.722481,44.492248,89.659690,E-COMMER,RUN.czC,RUN.czO,top4running.cz,2019-07-02 13:03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672599,2100363952,2021-09-13 00:00:00,CARD_GOPAY,S114,-1,NaN,31.043965,0.000000,31.043965,E-COMMER,RUN.skC,RUN.skO,top4running.sk,2021-09-13 13:20:00
1691358,2100373980,2021-09-20 00:00:00,CARD_GOPAY,S114,-1,NaN,16.343707,0.000000,16.343707,E-COMMER,RUN.skC,RUN.skO,top4running.sk,2021-09-20 17:56:00
1767684,2100418711,2021-10-19 00:00:00,GOPAY_CARD,S114,-1,NaN,3.203101,0.000000,3.217054,E-COMMER,RUN.czC,RUN.czO,top4running.cz,2021-10-19 21:25:00
1771959,2100421188,2021-10-21 00:00:00,COD,S114,-1,NaN,3.054984,0.000000,3.065891,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2021-10-21 16:42:00


It can be identified that in Orders table, products are presented individually (on 1 unique row), despite that an order can contain multiple products on it. For instance, the 3 first rows of the previous table are 3 different products (item_codes: ZB00018246, S101, and S113) that are part of the same order_id (order_id: 1900092622) 

##### 1.2. Items Table

In [29]:
items.head()

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
0,ZB00210807,Studio Metallic LS Top,51951501,86,Puma,200.0,Apparel,230.0,T-Shirts,233.0,Longsleeved (LS) shirts,fitness,women,adults,Black,L
1,ZB00210813,TRAIN TECH EVOKNIT SS TEE,52011101,86,Puma,200.0,Apparel,230.0,T-Shirts,0.0,NaN,fitness,men,adults,Black,S
2,ZB00210815,TRAIN TECH EVOKNIT SS TEE,52011123,86,Puma,200.0,Apparel,230.0,T-Shirts,0.0,NaN,fitness,men,adults,Orange,L
3,ZB00210821,TRAIN TECH EVOKNIT SS TEE,52011130,86,Puma,200.0,Apparel,230.0,T-Shirts,0.0,NaN,fitness,men,adults,Yellow,XL
4,ZB00261295,LIGA Baselayer Tee LS,655920-027,86,Puma,200.0,Apparel,230.0,T-Shirts,233.0,Longsleeved (LS) shirts,football,men,adults,Green,L


More relevant: item_code, brand_id, name, group1_id 

In [32]:
#Number of rows and columns
items.shape

(309128, 16)

In [33]:
#df info
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309128 entries, 0 to 309127
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   item_code  309128 non-null  object 
 1   item_name  306002 non-null  object 
 2   style      302331 non-null  object 
 3   brand_id   309128 non-null  int64  
 4   name       309128 non-null  object 
 5   group0_id  309093 non-null  float64
 6   group0     308302 non-null  object 
 7   group1_id  301709 non-null  float64
 8   group1     265348 non-null  object 
 9   group2_id  301709 non-null  float64
 10  group2     71773 non-null   object 
 11  category   269282 non-null  object 
 12  gender     301644 non-null  object 
 13  age        301562 non-null  object 
 14  color      263532 non-null  object 
 15  size       301087 non-null  object 
dtypes: float64(3), int64(1), object(12)
memory usage: 37.7+ MB


In [34]:
#Check for nulls
items.isnull().sum()

item_code         0
item_name      3126
style          6797
brand_id          0
name              0
group0_id        35
group0          826
group1_id      7419
group1        43780
group2_id      7419
group2       237355
category      39846
gender         7484
age            7566
color         45596
size           8041
dtype: int64

In [111]:
#Check for duplicates
items[items.duplicated()]

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size


There are no duplicates in items table.

In [114]:
#Top 10 brands
items['name'].value_counts().head(11)

Nike                 128931
adidas                54287
Under Armour          24202
Puma                  20169
Asics                  9485
adidas Originals       8274
- žádný výrobce -      7069
Salomon                5576
Reebok                 5514
VANS                   5416
Craft                  4505
Name: name, dtype: int64

žádný výrobce = sin fabricante (It's not a brand)

In [126]:
#Top 6 by product group
items['group0'].value_counts().head(6)

Footwear     150127
Apparel      137334
Equipment     13121
Other          7386
Nutrition       277
Services         57
Name: group0, dtype: int64

In [124]:
#Top 10 by product type
items['group1'].value_counts().head(10)

Running shoes     45942
Football shoes    40562
T-Shirts          39425
Other Footwear    35139
Pants             29492
Sweatshirts       16083
Jackets           10997
Jerseys            8741
Fitness Shoes      7569
Socks              6883
Name: group1, dtype: int64

#### 2. Data Cleaning

##### 2.1. Orders Table 

In [24]:
#Drop columns from orders table
orders_model = orders.drop(columns=['payment','gift_quantity','unit_rrp_vat_excl','department','channel','owner','CreatedAt']).rename(columns={"unit_price_vat_excl": "unit_price"})
#Convert string date to datetime objects
orders_model['date'] = pd.to_datetime(orders_model['date'])

In [25]:
orders_model

,order_id,date,item_code,quantity,unit_price,unit_cogs,site
0,2000093387,2020-04-24,S101,1,3.506048,0.000000,top4running.hu
1,2000093387,2020-04-24,S113,1,-0.705913,0.000000,top4running.hu
2,2000093388,2020-04-24,ZB00089178,1,5.829845,2.209302,11teamsports.cz
3,2000093388,2020-04-24,ZB00138060,1,5.829845,2.209302,11teamsports.cz
4,2000093388,2020-04-24,ZB00015664,1,19.379845,18.731008,11teamsports.cz
...,...,...,...,...,...,...,...
2128519,521008740,2021-10-11,S121,1,0.028682,0.000000,other
2128520,521009376,2021-10-26,S121,1,0.030620,0.000000,other
2128521,521009994,2021-11-11,S121,1,0.022481,0.000000,other
2128522,521011437,2021-12-23,S121,1,0.005039,0.000000,other


##### 2.2 Items Table

In [28]:
#Drop columns from items table
items_model = items.drop(columns=['style','group0_id','group1_id','group2_id','gender','age','color','size']).rename(columns={"name": "brand_name"})

In [40]:
#Fill null values
items_model['item_name'] = items_model['item_name'].fillna('Other')
items_model['group0'] = items_model['group0'].fillna('Other')
items_model['group1'] = items_model['group1'].fillna('Other')
items_model['group2'] = items_model['group2'].fillna('Other')
items_model['category'] = items_model['category'].fillna('Other')

##### 2.3 Merging orders_model and items_model dataframes

In [63]:
#Join orders_model and items_model dataframes
dataset = pd.merge(orders_model, items_model, how='inner', on ='item_code')

In [64]:
dataset.head()

,order_id,date,item_code,quantity,unit_price,unit_cogs,site,item_name,brand_id,brand_name,group0,group1,group2,category
0,2000093387,2020-04-24,S101,1,3.506048,0.0,top4running.hu,Dopravné,-1,- žádný výrobce -,Other,Other,Other,Other
1,2000093388,2020-04-24,S101,1,1.889922,0.0,11teamsports.cz,Dopravné,-1,- žádný výrobce -,Other,Other,Other,Other
2,2000093389,2020-04-24,S101,1,1.889922,0.0,11teamsports.cz,Dopravné,-1,- žádný výrobce -,Other,Other,Other,Other
3,2000093391,2020-04-24,S101,1,3.737403,0.0,11teamsports.sk,Dopravné,-1,- žádný výrobce -,Other,Other,Other,Other
4,2000093393,2020-04-24,S101,1,1.889922,0.0,top4sport.cz,Dopravné,-1,- žádný výrobce -,Other,Other,Other,Other


#### 3. Feature Engineering

In [65]:
#Create a column for country code
dataset['country'] = orders['site'].str[-2:]
#Replace 2 letters of the country for the country name
dataset['country'] = dataset['country'].replace({'at':'Austria', 'be':'Belgium', 'bg':'Bulgaria', 'cz':'Czech Republic', 'de':'Germany', 'dk':'Denmark', 'er':'Other', 'es':'Spain', 'fi':'Finland', 'fr':'France', 'hr':'Croatia', 'hu':'Hungary','ie':'Ireland', 'it':'Italy', 'nl':'Netherlands', 'pl':'Poland', 'om':'Multiple Countries', 'pt':'Portugal', 'ro':'Romania','se':'Sweden', 'si':'Slovenia', 'sk':'Slovakia', 'uk':'United Kingdom'})
dataset = dataset.drop(columns=['site'])

In [66]:
#Number of transaction per country
dataset['country'].value_counts()

Czech Republic        706536
Slovakia              357718
Hungary               285658
Romania               276361
Germany               122508
Spain                 119064
France                 72978
Italy                  41420
Croatia                37043
Multiple Countries     34394
Ireland                16581
Denmark                14015
Netherlands            10866
Austria                10738
Sweden                  7596
Portugal                4923
Poland                  4326
Belgium                 3393
Finland                 1601
Slovenia                 713
Other                     72
United Kingdom            14
Bulgaria                   6
Name: country, dtype: int64

The Top 6 countries in terms of number of transactions are Czech Republic, Slovakia, Hungary, Romania, Germany, and Spain.

In [ ]:
dataset[dataset['quantity']==-1]

##### 3.1 Financial Features

##### Sales / Revenue (total amount earned from sales)

* Gross Sales / Gross Revenue = Sum of all sales [(Sales price per unit) x (Total units sold)]

* Net sales / Net Revenue = Gross sales – Returns – Allowances – Discounts

##### Profit

* Gross Profit = Net Sales - COGS


*(Dataset contains information about shipping, discounts, and returned items)*

In [73]:
#Net Sales
dataset['net_sales'] = dataset['unit_price'] * dataset['quantity']

#Gross Profit
dataset['gross_profit'] = (dataset['unit_price'] - dataset['unit_cogs']) * dataset['quantity']

In [77]:
#Store dataset into CSV file
dataset.to_csv(r'data/dataset.csv', index=False, sep=',') 